 **Task description**

The goal is to analyze the database of the purchased service for reading books by subscription in order to further formulate a value proposition for a new product.

 **Goal:**

* get acquainted with the data stored in the database,
parse query results.

The data is arranged in 5 tables containing information about books, publishers, authors, user ratings and book reviews.

In [1]:
# import libraries
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# set configuration for database connection
db_config = {'user': 'praktikum_student', # user name
             'pwd': 'Sdf4$2;d-d30pp',     # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,                # connection port
             'db': 'data-analyst-final-project-db'}  # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

* **Study the tables (print the first rows)**

In [2]:
tables_list = ['books', 'authors', 'publishers', 'ratings', 'reviews']

In [3]:
for t in tables_list:
    print('Table {} head'.format(t))
    display(pd.io.sql.read_sql('''SELECT * FROM {} LIMIT 5'''.format(t), con = engine))
    print()

Table books head


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Table authors head


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Table publishers head


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Table ratings head


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Table reviews head


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Find the number of books released after January 1, 2000.**

In [4]:
query1 = """SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01'"""
pd.io.sql.read_sql(query1, con = engine)

,count
0,819


There were 819 books published after January 1, 2000.

**Find the number of user reviews and the average rating for each book.**

In [9]:
query2 = ''' SELECT title, cnt_review , avg_raiting
            FROM books 
            LEFT JOIN (SELECT book_id, COUNT(review_id) AS cnt_review 
                        FROM reviews
                        GROUP BY book_id) AS sub
            ON books.book_id = sub.book_id
            LEFT JOIN (SELECT book_id, AVG(rating) AS avg_raiting
                        FROM ratings 
                        GROUP BY book_id) AS sub_2
            ON books.book_id = sub_2.book_id
        '''
pd.io.sql.read_sql(query2, con = engine)

,title,cnt_review,avg_raiting
0,The Body in the Library (Miss Marple #3),2.0,4.500000
1,Galápagos,2.0,4.500000
2,A Tree Grows in Brooklyn,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,The Prophet,4.0,4.285714
...,...,...,...
995,Alice in Wonderland,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,Christine,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


**Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).**

In [6]:
query3 = """SELECT publisher, COUNT(book_id) as n_books 
            FROM books LEFt JOIN publishers ON books.publisher_id = publishers.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY n_books DESC
            LIMIT 1
            
            """
pd.io.sql.read_sql(query3, con = engine)

,publisher,n_books
0,Penguin Books,42


Penguin Books publisher is the most successfull publisher that released the biggest amount of books with more than 50 pages.

**Identify the author with the highest average book rating (look only at books with at least 50 ratings).**

In [7]:
query4 = """SELECT author, ROUND(AVG(avg_rating), 2) AS avg_rating
FROM (SELECT b.book_id, b.author_id, AVG(r.rating) AS avg_rating
  FROM books AS b 
  LEFT JOIN ratings AS r ON b.book_id = r.book_id 
  GROUP BY b.book_id 
  HAVING COUNT(r.rating_id) >= 50
  ORDER BY AVG(r.rating) DESC) AS sub
  LEFT JOIN authors AS a on sub.author_id = a.author_id
  GROUP BY author
  LIMIT 1;
"""
pd.io.sql.read_sql(query4, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


J.K.Rowling is one of the most rated authors with the highest average rating.

**Find the average number of text reviews among users who rated more than 50 books.**

In [8]:
query5 = """SELECT ROUND(AVG(n_reviews), 2) AS avg_reviews
            FROM
            (
            SELECT COUNT(rvw.review_id) AS n_reviews
                  FROM 
                  (SELECT username
                  FROM ratings 
                  GROUP BY username
                  HAVING COUNT(rating_id) > 50) AS usrs
            LEFT JOIN reviews AS rvw ON usrs.username = rvw.username
            GROUP BY usrs.username) AS s
            """
pd.io.sql.read_sql(query5, con = engine)

,avg_reviews
0,24.33


Active users (those, who rated more than 50 books) on average write 24 text reviews. 

In our research we concluded the next:
1. There were 819 books published after January 1, 2000.
2. The number of user reviews and the average rating for each book.
3. Penguin Books publisher is the most successfull publisher that released the biggest amount of books with more than 50 pages.
4. J.K.Rowling is one of the most rated authors with the highest average rating.
5. Active users (those, who rated more than 50 books) on average write 24 text reviews.